In [51]:
import requests as r
from flask import request 
from bs4 import BeautifulSoup as bs
# 用于伪造headers，访问者
from faker import Faker
import random

In [52]:
help(request)

Help on LocalProxy in module werkzeug.local object:

class LocalProxy(builtins.object)
 |  Acts as a proxy for a werkzeug local.  Forwards all operations to
 |  a proxied object.  The only operations not supported for forwarding
 |  are right handed operands and any kind of assignment.
 |  
 |  Example usage::
 |  
 |      from werkzeug.local import Local
 |      l = Local()
 |  
 |      # these are proxies
 |      request = l('request')
 |      user = l('user')
 |  
 |  
 |      from werkzeug.local import LocalStack
 |      _response_local = LocalStack()
 |  
 |      # this is a proxy
 |      response = _response_local()
 |  
 |  Whenever something is bound to l.user / l.request the proxy objects
 |  will forward all operations.  If no object is bound a :exc:`RuntimeError`
 |  will be raised.
 |  
 |  To create proxies to :class:`Local` or :class:`LocalStack` objects,
 |  call the object as shown above.  If you want to have a proxy to an
 |  object looked up by a function, you can (as

In [4]:
agent = Faker('zh-CN').user_agent()
def get_cityCode(province='',city=''):
    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
    result = r.get(city_json,
                headers={'User-Agent':agent}).json()
    if (result['message'] == 'Success'):
        cityList = result['zpData']['cityList']
        for p in cityList:
            if p['name'] == province:
                sub = p['subLevelModelList']
                flag = 1
                for c in sub:
                    if city == c['name']:
                        return p['code'],c['code']  
                return p['code']
        else:
            print('找不到对象')
            return None
    
    else:
        print('调用boss直聘的获取地区编码的api有问题！',city)

In [47]:
city = get_cityCode('广东','广州')[1]

# 数据上限399条
def get_job(job,city):
    proxies = [{'http':'http://47.107.175.190:8000'},
          {'http':'http://163.204.242.64:9999'},
          {'http':'http://47.112.214.45:8000'},
        {'http':'http://163.204.242.181:9999'},
        {'http':'http://163.204.241.214:9999'}]
    
    job_json = 'https://www.zhipin.com/mobile/jobs.json'
    params = {
        'query':job,
        'page':1,
        'city':city
    }
    job_data = []
    while 1:
        pro = random.choice(proxies)
        response = r.get(job_json,
              params=params,
              headers={'User-Agent':agent},
            proxies=pro).json()
        
        try:
            
            if response['html']:
                params['page'] +=1
                html = bs(response['html'],'html.parser').find_all(class_='item')
                for h in html:
                # item字典一定要写在循环里面且每遍历完一个职位条目就重新清空字典、保存在job列表当中
                # 否则会重复将最后的值，反复追加到job列表，可见变量名只是个字典的映射。
                    item = {}
                    item['salary'] = h.find('span',"salary").string 
                    item['company'] = h.find('div','name').string
                    msg = h.find('div','msg').find_all('em')
                    for em in range(3):
                        item['city'] = msg[0].string
                        item['experience'] = msg[1].string
                        item['education'] = msg[2].string
                    job_data.append(item)
            elif response['resmsg'] != '成功':
                print('调用boss直聘获取工作列表api有问题！或者是网络、服务器有问题')
                continue
            elif response['html'] == '' and params['page']==1:
                print('用户输入的关键词有误，找不到对象！')
                return None
                break
            else:
                break
        # 返回的json数据结果没有html键，说明ip已被封
        except KeyError:
            print(response)
            break
        # 连接代理服务出现异常时，就把那个代理ip从列表中删除并继续用另一个代理ip
        except (ConnectTimeout,ProxyError,ReadTimeout):
            proxies.remove(pro)
            continue
    return job_data


In [6]:
def count_num(data,key='education'):
    edu = {}
    length = len(data)
    for j in range(length):
        k=data[j][key]
        edu[k] = edu.get(k,0)+1
    return sorted(edu.items(),
                  key=lambda x : x[1],
                  reverse=True)

In [50]:
proxies = [{'http':'http://47.107.175.190:8000'},
      {'http':'http://163.204.242.64:9999'},
      {'http':'http://47.112.214.45:8000'},
    {'http':'http://163.204.242.181:9999'},
    {'http':'http://163.204.241.214:9999'}]

job_json = 'https://www.zhipin.com/mobile/jobs.json'

params = {
    'query':'jfdklgjflkgj',
    'page':1,
    'city':city
}
pro = random.choice(proxies)
response = get_job('jfdklgjfl',city = get_cityCode('广东','广州')[1])

print(response)

用户输入的关键词有误，找不到对象！
None


In [45]:
response['html'] == ''

True

In [11]:
from pyecharts import options as opts
from pyecharts.charts import Pie,Bar,Map,Tab
from pyecharts.faker import Faker

In [25]:
def pie_chart(data,title,seriers_name,width=900,height=500):
    try:
        w = str(width)+'px'
        h = str(height)+'px'
        c = (
            Pie(init_opts=opts.InitOpts(width=w,height=h))
            .add(seriers_name,data)
            .set_global_opts(title_opts=opts.TitleOpts(title=title))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
        )
        return c
    except ValueError:
        print("请输入正确的数据格式！比如：[('本科', 290), ('大专', 88), ('学历不限', 16), ('高中', 5), ('中专/中技', 1)]")
edu_data = count_num(job,'education')
pie_chart(edu_data,'产品经理的学历要求','该学历要求的数量统计').render_notebook()

In [31]:
def bar_chart(ex_data,title='',seriers_name='',color='red',width=900,height=500):
    xaxis = list(dict(ex_data).keys())
    yaxis = list(dict(ex_data).values())
    w = str(width)+'px'
    h = str(height)+'px'
    c = (
        Bar(init_opts=opts.InitOpts(width=w,height=h))
        .add_xaxis(xaxis)
        .add_yaxis(seriers_name,yaxis,color=color)
  #      .set_series_opts(label_opts=opts.LabelOpts(color='blue'))
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            datazoom_opts=opts.DataZoomOpts()
        )
    )
    return c
exnum = count_num(job,'experience')
bar_chart(exnum,color='black').render_notebook()

In [30]:
def divid_salary(data):
    # 依据个人所得税税率表来将salary数据分组，否则太多太杂了。
    # 以月薪、千元为单位
    if isinstance(data,dict):
        sk = {'<3k':0,'3-12k':0,'12-25k':0,'25-35k':0,'35-55k':0,'55k-80k':0,'>80k':0}
        for s in data:
            # 由于我们的产品目标是为职场小白提供择业意见，小白往往只能获得最低工资，所以我们仅选取了最低工资，方便将每个工资范围进行分组
            sl = s.split('-')
            if sl[1][-1] == 'K':
                key = int(sl[0])
                if key <=3:
                    sk['<3k'] += data[s]
                elif key > 3 and key <= 12:
                    sk['3-12k'] += data[s]
                elif key > 12 and key <= 25:
                    sk['12-25k'] += data[s]
                elif key > 25 and key <= 35:
                    sk['25-35k'] += data[s]
                elif key > 35 and key <= 55:
                    sk['35-55k'] += data[s]
                elif key > 55 and key <= 80:
                    sk['55-80k'] += data[s]
                elif key > 80:
                    sk['>80k'] +=salary[s]
        result = sorted(sk.items(),key=lambda x:x[1],reverse=True)
        return result
    else:
        print('请输入字典类型的数据。')
sa = divid_salary(dict(count_num(job,'salary')))
bar_chart(sa,'产品经理薪资水平统计','该薪资水平的数量统计',color='red').render_notebook()

In [29]:

# 要单独将地图可视化放到另一个页面才行。
def get_mapData(jobName):
    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
    adv = r.get(city_json,
                    headers={'User-Agent':agent}).json()['zpData']['cityList']

    # 创建一个市级行政区域与省级行政区的映射，方便后面统计各省的职位数量
    pt = {}
    for ad in adv:
        for a in ad['subLevelModelList']:
            pt[a['name']] = ad['name']

    # 先统计每个城市的职位数量，再统计省级行政区的职位数量
    pycity = count_num(get_job(jobName,100010000),'city')
    map_data = {}
    for p in pycity:
        key = pt[p[0]]
        map_data[key]= map_data.get(key,0)+p[1]  
    return map_data.items()

def map_chart(data,series_name,title,width=900,height=500):
    mapData = list(data)
    w = str(width)+'px'
    h = str(height)+'px'
    c = (
        Map(init_opts=opts.InitOpts(width=w,height=h))
        .add(series_name,mapData)
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            visualmap_opts=opts.VisualMapOpts(max_= mapData[0][1])
        )
    )  
    return c

map_chart(get_mapData('产品经理'),'该岗位的需求数量：','产品经理岗位需求').render_notebook()

In [37]:
salary = dict(count_num(job,'salary'))
sa_bar = bar_chart(divid_salary(salary),'产品经理薪资水平统计','该薪资水平的数量统计',color='red')
ex_bar = bar_chart(count_num(job,'experience'),color='black')
edu_pie = pie_chart(count_num(job,'education'),'产品经理的学历要求','该学历要求的数量统计')

def tab_charts(chart_list,name_list):
    tab = Tab()
    count = 1
    for cl in chart_list:
        tab.add(cl,count)
        count +=1
    return tab
tab_charts([sa_bar,ex_bar,edu_pie]).render_notebook()

In [40]:
def test():
    global x
    x = 'l am trying'

In [41]:
test()
print(x)

l am trying
